In [1]:
# data
# model
# loss
# eval
# infer


In [2]:
# 目标是先建立一个非常强的baseline；数据端先简易来；模型端好好弄，loss好好弄， eval好好弄;
1. infer ? 
2. eval mAP?

In [5]:
![image.png](attachment:image.png)

/bin/sh: 1: Syntax error: word unexpected (expecting ")")


In [1]:
import cv2
import os
import torch
import json
import ipdb
import numpy as np
from torchvision import models
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt
from data import Jiu, Jiu2, Jiu3, do_resize
from model import *
from loss import multi_class_entropy, lovasz_softmax
from fastprogress.fastprogress import master_bar, progress_bar
from eval import mAP
from timeit import default_timer as timer
from visdom import Visdom

%matplotlib inline
import ipdb


def time_to_str(t, mode='min'):
    from timeit import default_timer as timer
    
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError
        

def train(loader, model, dataset, device):
    model = model.to(device)
    running_loss = 0.
    model.train()
    for imgs, masks, cls, bbox, H, W in progress_bar(loader, parent=mb):
        optimizer.zero_grad()
        imgs, masks = imgs.to(device), masks.to(device)
        with torch.set_grad_enabled(True):
            logits = model(imgs)
            loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
        running_loss += loss.item()*imgs.size(0)
    return running_loss/len(dataset)


def valid(loader, model, dataset, device):
    model = model.to(device)
    running_loss = 0.
    model.eval()
    for imgs, masks, cls, bbox, H, W in progress_bar(loader, parent=mb):    
#     for imgs, masks, cls, bbox, H, W in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        with torch.no_grad():
            logits = model(imgs)
            loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
            loss = loss1 + loss2
        running_loss += loss.item()*imgs.size(0)
    vis.images(imgs[:5], nrow=5, win='valid_imgs',opts={'title':'imgs'})
#     vis.images(masks[:5], nrow=5, win='valid_masks',opts={'title':'masks'})
    return running_loss/len(dataset)


weight_dic = {1:0.15,
2:0.09,
3:0.09,
4:0.05,
5:0.13,
6:0.05,
7:0.12,
8:0.13,
9:0.07,
10:0.12}
   

In [2]:
batch_size = 20
model = Unet()
lr = 0.1
min_lr = 0.0001
EPOCH=200
snapshot = 4
scheduler_step = EPOCH//snapshot
device = 'cuda'
fold_id = 1

train_data = Jiu3(fold_id=fold_id, mode='train')
train_loader = DataLoader(
                    train_data,
                    shuffle=RandomSampler(train_data),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

valid_data = Jiu3(fold_id=fold_id, mode='valid')
valid_loader = DataLoader(
                    valid_data,
                    shuffle=RandomSampler(valid_data),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

In [3]:
# !python -m visdom.server

In [10]:
vis = Visdom(env='jiu3jiu')

Setting up a new session...


In [5]:
optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
start = timer()
mb = master_bar(range(EPOCH))

for epoch in mb:
    scheduler.step()
    epoch_loss = train(train_loader, model, train_data, device)
    epoch_val_loss = valid(valid_loader, model, valid_data, device)
    print('epoch:{} time:{} epoch_train_loss:{} epoch_valid_loss:{}'.format(epoch, time_to_str((timer() - start), 'min'), epoch_loss, epoch_val_loss))
    vis.line(X=[epoch], Y=[[epoch_loss, epoch_val_loss]], opts=dict(markers=False), win='loss', update='append' if epoch>0 else None)

    # 重置lr;
    if (epoch + 1) % scheduler_step == 0:
        torch.save(model.state_dict(),  'unet_'+ str(epoch) + '.pth')
        optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
        scheduler.step()
        

/home/qibo/qb_vir_env/lib/python3.6/site-packages/torch/nn/functional.py:2457: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


epoch:0 time: 0 hr 03 min epoch_train_loss:1.0986896159410304 epoch_valid_loss:1.032332632763082
epoch:1 time: 0 hr 07 min epoch_train_loss:1.0044096020848912 epoch_valid_loss:0.9998582649608582
epoch:2 time: 0 hr 11 min epoch_train_loss:0.9795404288990194 epoch_valid_loss:0.9659284175600913
epoch:3 time: 0 hr 15 min epoch_train_loss:0.962783264719041 epoch_valid_loss:0.9529896476785428


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/qibo/qb_vir_env/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 21, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/qibo/qb_vir_env/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 276, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)


KeyboardInterrupt: 